In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

review = ["This film was a waste of time","This film was very nice"]

result = classifier(review)

print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9998158812522888}, {'label': 'POSITIVE', 'score': 0.9998468160629272}]


In [17]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

reviews = ["This film was a waste of time","This film was very nice"]

tokenizer = classifier.tokenizer

result = tokenizer(review, padding = True, return_tensors = "pt")

#print(result)

print(result['input_ids'])

print(result['attention_mask'])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


tensor([[ 101, 2023, 2143, 2001, 1037, 5949, 1997, 2051,  102],
        [ 101, 2023, 2143, 2001, 2200, 3835,  102,    0,    0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [18]:
model = classifier.model

outputs = model(result['input_ids'],result['attention_mask'])

print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.7709, -3.8290],
        [-4.2027,  4.5809]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [19]:
import torch

probability_dist = torch.nn.functional.softmax(outputs.logits, dim=1)

print(probability_dist)

tensor([[9.9982e-01, 1.8409e-04],
        [1.5321e-04, 9.9985e-01]], grad_fn=<SoftmaxBackward0>)


In [21]:
class_labels = ["Negative", "Positive"]

# Convert predictions into human-readable format
for i, review in enumerate(reviews):
    negative_score = probability_dist[i, 0].item()
    positive_score = probability_dist[i, 1].item()
    predicted_label = class_labels[positive_score > negative_score]  # Choose label based on max score

    print(f"Review: \"{review}\"")
    print(f"Predicted Sentiment: {predicted_label} (Positive: {positive_score:.4f}, Negative: {negative_score:.4f})\n")

Review: "This film was a waste of time"
Predicted Sentiment: Negative (Positive: 0.0002, Negative: 0.9998)

Review: "This film was very nice"
Predicted Sentiment: Positive (Positive: 0.9998, Negative: 0.0002)



In [22]:
from transformers import pipeline
import torch

# Initialize sentiment analysis pipeline with a pre-trained DistilBERT model
sentiment_pipeline = pipeline(
    task="sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

# List of movie reviews for sentiment analysis
reviews = ["This film was a waste of time", "This film was very nice"]

# Extract the tokenizer from the pipeline
tokenizer = sentiment_pipeline.tokenizer

# Tokenize the input reviews and convert them to tensors
tokenized_inputs = tokenizer(reviews, padding=True, return_tensors="pt")

print(tokenized_inputs)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


{'input_ids': tensor([[ 101, 2023, 2143, 2001, 1037, 5949, 1997, 2051,  102],
        [ 101, 2023, 2143, 2001, 2200, 3835,  102,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])}


In [24]:
# Extract the model from the pipeline
sentiment_model = sentiment_pipeline.model

# Perform model inference using tokenized inputs
model_outputs = sentiment_model(
    input_ids=tokenized_inputs["input_ids"],
    attention_mask=tokenized_inputs["attention_mask"]
)

print(model_outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.7709, -3.8290],
        [-4.2027,  4.5809]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [25]:
# Apply softmax to convert logits into probabilities
predicted_probabilities = torch.nn.functional.softmax(model_outputs.logits, dim=1)

# Define labels for the classes (Index 0 = Negative, Index 1 = Positive)
class_labels = ["Negative", "Positive"]

# Convert predictions into human-readable format
for i, review in enumerate(reviews):
    negative_score = predicted_probabilities[i, 0].item()
    positive_score = predicted_probabilities[i, 1].item()

    # Determine predicted label based on highest probability
    predicted_label = class_labels[0] if negative_score > positive_score else class_labels[1]

    print(f"Review: \"{review}\"")
    print(f"Predicted Sentiment: {predicted_label} (Positive: {positive_score:.4f}, Negative: {negative_score:.4f})\n")


Review: "This film was a waste of time"
Predicted Sentiment: Negative (Positive: 0.0002, Negative: 0.9998)

Review: "This film was very nice"
Predicted Sentiment: Positive (Positive: 0.9998, Negative: 0.0002)



In [31]:
from transformers import pipeline
import torch

sentiment_pipeline = pipeline(
    task="sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)


tokenizer = sentiment_pipeline.tokenizer

print(tokenizer("Hello my name is Sumit"))

tokens = tokenizer.tokenize("Hello my name is Sumit")
print(tokens)

token_ids = tokenizer.encode("Hello my name is Sumit")
print(token_ids)

decoded_tokens = tokenizer.decode(token_ids)
print(decoded_tokens)

tokenizer.get_vocab()

tokenizer.vocab_size

Device set to use cpu


{'input_ids': [101, 7592, 2026, 2171, 2003, 7680, 4183, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
['hello', 'my', 'name', 'is', 'sum', '##it']
[101, 7592, 2026, 2171, 2003, 7680, 4183, 102]
[CLS] hello my name is sumit [SEP]


30522

In [32]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)

print(tokenizer("Hello my name is Sumit"))



{'input_ids': [101, 7592, 2026, 2171, 2003, 7680, 4183, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# List of movie reviews for sentiment analysis
reviews = ["This film was a waste of time", "This film was very nice"]

# Extract the tokenizer from the pipeline
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the input reviews and convert them to tensors
tokenized_inputs = tokenizer(reviews, padding=True, return_tensors="pt")

# Extract the model from the pipeline
sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Perform model inference using tokenized inputs
model_outputs = sentiment_model(
    input_ids=tokenized_inputs["input_ids"],
    attention_mask=tokenized_inputs["attention_mask"]
)

# Apply softmax to convert logits into probabilities
predicted_probabilities = torch.nn.functional.softmax(model_outputs.logits, dim=1)

# Define labels for the classes (Index 0 = Negative, Index 1 = Positive)
class_labels = ["Negative", "Positive"]

# Convert predictions into human-readable format
for i, review in enumerate(reviews):
    negative_score = predicted_probabilities[i, 0].item()
    positive_score = predicted_probabilities[i, 1].item()

    # Determine predicted label based on highest probability
    predicted_label = class_labels[0] if negative_score > positive_score else class_labels[1]

    print(f"Review: \"{review}\"")
    print(f"Predicted Sentiment: {predicted_label} (Positive: {positive_score:.4f}, Negative: {negative_score:.4f})\n")


Review: "This film was a waste of time"
Predicted Sentiment: Negative (Positive: 0.0002, Negative: 0.9998)

Review: "This film was very nice"
Predicted Sentiment: Positive (Positive: 0.9998, Negative: 0.0002)



In [34]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
import torch

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# List of movie reviews for sentiment analysis
reviews = ["This film was a waste of time", "This film was very nice"]

# Extract the tokenizer from the pipeline
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

# Tokenize the input reviews and convert them to tensors
tokenized_inputs = tokenizer(reviews, padding=True, return_tensors="pt")

# Extract the model from the pipeline
sentiment_model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Perform model inference using tokenized inputs
model_outputs = sentiment_model(
    input_ids=tokenized_inputs["input_ids"],
    attention_mask=tokenized_inputs["attention_mask"]
)

# Apply softmax to convert logits into probabilities
predicted_probabilities = torch.nn.functional.softmax(model_outputs.logits, dim=1)

# Define labels for the classes (Index 0 = Negative, Index 1 = Positive)
class_labels = ["Negative", "Positive"]

# Convert predictions into human-readable format
for i, review in enumerate(reviews):
    negative_score = predicted_probabilities[i, 0].item()
    positive_score = predicted_probabilities[i, 1].item()

    # Determine predicted label based on highest probability
    predicted_label = class_labels[0] if negative_score > positive_score else class_labels[1]

    print(f"Review: \"{review}\"")
    print(f"Predicted Sentiment: {predicted_label} (Positive: {positive_score:.4f}, Negative: {negative_score:.4f})\n")


Review: "This film was a waste of time"
Predicted Sentiment: Negative (Positive: 0.0002, Negative: 0.9998)

Review: "This film was very nice"
Predicted Sentiment: Positive (Positive: 0.9998, Negative: 0.0002)

